In [1]:
import os
import sys
import time

os.chdir('../' )
cwd = os.getcwd()
sys.path.append(cwd)

from oligo_designer_toolsuite.IO._database import CustomDB
from oligo_designer_toolsuite.oligo_selection._generate_probesets_h import ProbesetGenerator as ProbesetGenerator_h
from oligo_designer_toolsuite.oligo_selection._generate_probesets import ProbesetGenerator
from oligo_designer_toolsuite.oligo_efficiency import PadlockProbeScoring, PadlockSetScoring, AverageSetScoring
from oligo_designer_toolsuite.oligo_selection import padlock_heuristic_selection
from oligo_designer_toolsuite.experiment_specific import PadlockSequenceDesigner

from Bio.SeqUtils import MeltingTemp as mt

os.chdir('../' )
cwd = os.getcwd()
dir_output  = os.path.join(cwd, 'oligo-designer-toolsuite/output')
print(dir_output)



/home/francesco/Desktop/Work/oligo-designer-toolsuite/output


Initilitze the classes

In [2]:
dir_annotation = '/home/francesco/Desktop/Work/NCBI'
annotation = dir_annotation+'/GCF_000001405.40_GRCh38.p14_genomic.gtf'
sequence = dir_annotation+'/GCF_000001405.40_GRCh38.p14_genomic.fna'
file_oligos = '/home/francesco/Desktop/Work/oligo-designer-toolsuite/output/specificity_filter/oligo_DB_human_GRCh38_NCBI_release_current_gene_transcript.tsv'
#file_oligos = "/home/francesco/Desktop/Work/oligo-designer-toolsuite/output/specificity_filter/probes_info.txt"

database = CustomDB(probe_length_min = 30, probe_length_max = 40, dir_output=dir_output, file_annotation=annotation, file_sequence=sequence)
database.read_oligos_DB(format = 'tsv', file_oligos_DB_tsv = file_oligos)
#enalrge teh DB
genes = list(database.oligos_DB.keys())
"""for gene in genes:
    for i in range(1):
        database.oligos_DB[gene+f'_{i}'] = database.oligos_DB[gene]"""
padlock_scoring = PadlockProbeScoring(Tm_min = 52, Tm_opt = 60, Tm_max = 67, GC_content_min = 40, GC_content_opt = 50, GC_content_max = 60)
set_scoring = PadlockSetScoring()
#set_scoring = AverageSetScoring()
n_jobs = 1

/home/francesco/Desktop/Work/oligo-designer-toolsuite/oligo_designer_toolsuite/IO/_database.py:64: UserWarning: Species not specified.
  warnings.warn("Species not specified.")
/home/francesco/Desktop/Work/oligo-designer-toolsuite/oligo_designer_toolsuite/IO/_database.py:68: UserWarning: Genome assembly not specified.
  warnings.warn(f"Genome assembly not specified.")
/home/francesco/Desktop/Work/oligo-designer-toolsuite/oligo_designer_toolsuite/IO/_database.py:72: UserWarning: Annotation release not specified.
  warnings.warn(f"Annotation release not specified.")
/home/francesco/Desktop/Work/oligo-designer-toolsuite/oligo_designer_toolsuite/IO/_database.py:76: UserWarning: Annotation source not specified.
  warnings.warn(f"Annotation source not specified.")


In [7]:
probeset_generator = ProbesetGenerator(probeset_size = 5, 
                                        min_probeset_size= 2,
                                        probes_scoring = padlock_scoring, 
                                        set_scoring=set_scoring,
                                        heurustic_selection=padlock_heuristic_selection
                                    )
start = time.time()
database = probeset_generator.apply(database, n_sets=100, n_jobs=n_jobs)
print(f'time to process: {time.time() - start}')
database.write_probesets(dir_probesets='probesets')
for key in database.probesets.keys():
    print(f"{key}: {database.probesets[key].iloc[0]['set_score_0']}, nsets: {len(database.probesets[key])}")

time to process: 0.37003397941589355
AGRN: 0.8988571428571426, nsets: 40
TTLL10: 0.8430000000000005, nsets: 10
UBE2J2: 0.42571428571428527, nsets: 48
MIB2: 0.4742857142857143, nsets: 100


In [10]:
probeset_generator = ProbesetGenerator_h(probeset_size = 5, 
                                        min_probeset_size= 2,
                                        probes_scoring = padlock_scoring, 
                                        set_scoring=set_scoring,
                                        sort_ascending=True,
                                    )
start = time.time()
database.read_oligos_DB(format = 'tsv', file_oligos_DB_tsv = file_oligos)
database = probeset_generator.apply(database, n_sets=100, n_jobs=n_jobs, tries = 400, tries_decade=1e-1)
print(f'time to process: {time.time() - start}')
database.write_probesets(dir_probesets='probesets_heuristic')
for key in database.probesets.keys():
    print(f"{key}: {database.probesets[key].iloc[0]['set_score_0']}, nsets: {len(database.probesets[key])}")

time to process: 27.360677003860474
AGRN: 0.8988571428571426, nsets: 40
TTLL10: 1.0032857142857141, nsets: 72
UBE2J2: 0.40157142857142863, nsets: 100
MIB2: 0.5587142857142859, nsets: 100


time to process: 7.8144309520721436
AGRN: 0.8988571428571426
TTLL10: 1.0032857142857141
UBE2J2: 0.5674285714285711
MIB2: 0.558714285714285

In [5]:
import numpy as np
a = np.random.rand(5) > 0.3
print(a)
b = np.where(a)[0]
print(b)
for elem in b:
    print(elem)
print(len(b))

[ True False  True  True  True]
[0 2 3 4]
0
2
3
4
4


Now we create the  padlock probes probes

In [6]:

# define the necessary parameters
Tm_parameters = {
"check": True,
"strict": True,
"c_seq": None,
"shift": 0,
"nn_table": getattr(mt, "DNA_NN3"),
"tmm_table": getattr(mt, "DNA_TMM1"),
"imm_table": getattr(mt, "DNA_IMM1"),
"de_table": getattr(mt, "DNA_DE1"),
"dnac1": 50,  # [nM]
"dnac2": 0,
"selfcomp": False,
"dNTPs": 0,
"saltcorr": 7,
"Na": 39,  # [mM]
"K": 0,  # [mM]
"Tris": 0,  # [mM]
"Mg": 0,  # [mM]
}

Tm_correction_parameters = {
"DMSO": 0,
"DMSOfactor": 0.75,
"fmdfactor": 0.65,
"fmdmethod": 1,
"GC": None,
"fmd": 30,
}

detect_oligo_length_min = 18 #min length of detection oligo
detect_oligo_length_max = 25 #max length of detection oligo
detect_oligo_Tm_opt = 32

padlock_sequence_designer = PadlockSequenceDesigner(detect_oligo_length_min, detect_oligo_length_max, detect_oligo_Tm_opt, Tm_parameters, Tm_correction_parameters)
padlock_sequence_designer.design_padlocks(database)